In [1]:
import pandas as pd
import pyperclip
import keyboard
import time
import os
import threading
from openpyxl import load_workbook

In [ ]:
# Flag to control the script execution
running = True

def quit_handler():
    """Function to handle the 'q' key press"""
    global running
    running = False
    print("\nQuitting the script...")

def listen_for_quit_key():
    """Thread function to listen for the 'q' key"""
    keyboard.add_hotkey('q', quit_handler)
    keyboard.wait()

def translate_text(text_to_translate):
    """Helper function to translate a single text using DeepL via keyboard navigation"""
    if pd.isna(text_to_translate) or text_to_translate.strip() == "":
        return ""
        
    # Copy text to translate to clipboard
    pyperclip.copy(text_to_translate)
    
    # Focus on input area (assuming DeepL is active window)
    # Clear previous text with Ctrl+A and Delete
    keyboard.press_and_release('ctrl+a')
    time.sleep(0.3)
    keyboard.press_and_release('delete')
    time.sleep(0.3)
    
    # Paste new text
    keyboard.press_and_release('ctrl+v')
    
    # Wait for translation
    wait_time = 1.5  # Adaptive wait time
    print(f"Waiting for translation ({wait_time:.1f} seconds)...")
    time.sleep(wait_time)
    
    # Navigate to output area (typically Tab key works in DeepL)
    for _ in range(3):
        keyboard.press_and_release('tab')
        time.sleep(0.1)
    
    # Select all text in output area
    keyboard.press_and_release('ctrl+a')
    time.sleep(0.6)
    
    # Copy translated text
    keyboard.press_and_release('ctrl+c')
    time.sleep(0.6)
    
    # Get translated text from clipboard
    translated_text = pyperclip.paste()
    
    # Navigate back to input area for next iteration
    for _ in range(3):
        keyboard.press_and_release('shift+tab')
        time.sleep(0.1)
    
    return translated_text

def main():
    global running
    
    # Configuration
    source_file = "datas7.xlsx" #Beyler bu kısımda dönüştürmemiz gerekn original verilerin olduğu dökümanın uzantısını yazıcaksınız
    target_file = "grammer_mertcan7.xlsx" #Beyler bu kısımda kopyalanıcak dosyanın uzantısı olcak
    source_sheet = "Sheet1"
    target_sheet = "Sheet1"
    source_columns = [0, 1]  # Column indices (0 and 1 for first and second columns)
    target_columns = [0, 1]  # Target column indices
    
    # Read the source Excel file
    try:
        df = pd.read_excel(source_file, sheet_name=source_sheet)
        print(f"Successfully loaded source file with {len(df)} rows")
    except Exception as e:
        print(f"Error reading source file: {e}")
        return
    
    # Create target file if it doesn't exist
    if not os.path.exists(target_file):
        df_target = pd.DataFrame()
        df_target.to_excel(target_file, sheet_name=target_sheet, index=False)
        print(f"Created new target file: {target_file}")
    
    # Setup instructions
    print("\nKEYBOARD NAVIGATION SETUP:")
    print("1. Open DeepL desktop application")
    print("2. Make sure it's the active window")
    print("3. Input area should be selected and ready for text")
    print("\nPress 'q' at any time to quit the script")
    print("\nScript will start in 10 seconds...")
    time.sleep(10)
    
    # Process each row
    for index, row in df.iterrows():
        # Check if we should stop
        if not running:
            print("Script stopped by user")
            return
            
        try:
            print(f"\nProcessing row {index+1}:")
            
            # Load workbook for each row to make sure we have the latest version
            wb = load_workbook(target_file)
            ws = wb[target_sheet]
            
            # Process each column
            for col_idx, source_col in enumerate(source_columns):
                # Check if we should stop
                if not running:
                    print("Script stopped by user")
                    return
                    
                target_col = target_columns[col_idx]
                
                # Get the text to translate
                text_to_translate = str(row.iloc[source_col])
                if pd.isna(text_to_translate) or text_to_translate.strip() == "":
                    print(f"  Column {source_col+1}: Empty cell, skipping")
                    continue
                    
                print(f"  Column {source_col+1}: Translating: {text_to_translate[:30]}...")
                
                # Translate the text
                translated_text = translate_text(text_to_translate)
                print(f"  Translation: {translated_text[:30]}...")
                
                # Excel rows are 1-indexed, and we need to account for header
                row_num = index + 2
                
                # Convert column index to Excel column letter
                column_letter = chr(65 + target_col)
                cell = f"{column_letter}{row_num}"
                
                # Write translated text
                ws[cell] = translated_text
                print(f"  Saved translation to cell {cell}")
            
            # Save after processing both columns for this row
            wb.save(target_file)
            
            # Small delay between rows
            time.sleep(1.5)
            
        except Exception as e:
            print(f"Error processing row {index+1}: {e}")
    
    print("Translation complete!")

if __name__ == "__main__":
    print("DeepL Excel Translator using Keyboard Navigation")
    print("-----------------------------------------------")
    print("Press 'q' at any time to stop the script")
    
    try:
        # Start the keyboard listener in a separate thread
        listener_thread = threading.Thread(target=listen_for_quit_key)
        listener_thread.daemon = True
        listener_thread.start()
        
        # Run the main function
        main()
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    # Make sure the script doesn't exit immediately if it's a short run
    if running:
        print("\nScript finished execution")

DeepL Excel Translator using Keyboard Navigation
-----------------------------------------------
Press 'q' at any time to stop the script
Successfully loaded source file with 127 rows

KEYBOARD NAVIGATION SETUP:
1. Open DeepL desktop application
2. Make sure it's the active window
3. Input area should be selected and ready for text

Press 'q' at any time to quit the script

Script will start in 10 seconds...

Processing row 1:
  Column 1: Translating: I need to print 2 vector in op...
Waiting for translation (1.5 seconds)...
  Translation: I need to print 2 vectors in o...
  Saved translation to cell A2
  Column 2: Translating: You cannot pass parameters to ...
Waiting for translation (1.5 seconds)...
  Translation: You cannot pass parameters to ...
  Saved translation to cell B2

Processing row 2:
  Column 1: Translating: How do I find a substring from...
Waiting for translation (1.5 seconds)...
  Translation: How do I find a substring from...
  Saved translation to cell A3
  Column 2


Quitting the script...

Quitting the script...
